In [1]:
import netpixi
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('reviews_july.csv', sep=',')

In [3]:
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,73579864,brazilian,"I'll only speak about performance, since we al...",2020-07-31 22:19:22,2020-07-31 22:19:22,True,0,0,...,True,False,False,76561198074616040,148,12,29259.0,0.0,18588.0,1.597875e+09
1,292030,The Witcher 3: Wild Hunt,73576952,brazilian,Não tenho palavras para dizer como esse jogo e...,2020-07-31 20:46:46,2020-07-31 20:48:04,True,1,0,...,True,False,False,76561198120075031,38,9,4389.0,0.0,4389.0,1.557089e+09
2,292030,The Witcher 3: Wild Hunt,73575959,brazilian,Epic,2020-07-31 20:17:38,2020-07-31 20:17:38,True,0,0,...,True,False,False,76561198101494872,25,1,6653.0,0.0,4136.0,1.608502e+09
3,292030,The Witcher 3: Wild Hunt,73575666,brazilian,"Melhor jogo do fucking universo, compre isso e...",2020-07-31 20:09:24,2020-07-31 20:09:24,True,1,0,...,True,False,False,76561198032616918,80,2,24894.0,0.0,18568.0,1.608581e+09
4,292030,The Witcher 3: Wild Hunt,73574848,brazilian,Um dos melhores games já feito.,2020-07-31 19:45:16,2020-07-31 19:45:16,True,1,0,...,True,False,False,76561198276339634,37,4,7796.0,0.0,7755.0,1.596227e+09


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80068 entries, 0 to 80067
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   app_id                          80068 non-null  int64  
 1   app_name                        80068 non-null  object 
 2   review_id                       80068 non-null  int64  
 3   language                        80068 non-null  object 
 4   review                          80025 non-null  object 
 5   timestamp_created               80068 non-null  object 
 6   timestamp_updated               80068 non-null  object 
 7   recommended                     80068 non-null  bool   
 8   votes_helpful                   80068 non-null  int64  
 9   votes_funny                     80068 non-null  int64  
 10  weighted_vote_score             80068 non-null  float64
 11  comment_count                   80068 non-null  int64  
 12  steam_purchase                  

- Criando coluna para variável de Controle


In [5]:
df['level_of_positivite'] = df.groupby('author.steamid')['recommended'].transform(lambda x: x.sum() / x.count())

In [6]:
def get_edges(df):
    users = df['author.steamid'].unique()
    
    edges = []
    for user in users:
        games = df[df['author.steamid']==user]['app_name'].unique()
        for game in games:
            users_game = df[df['app_name']==game]['author.steamid'].unique()
            for user_game in users_game:
                if user_game != user:
                    edges.append(tuple(sorted([user, user_game])))
    
    edges = np.array(edges)

    edges, counts = np.unique(edges, axis=0, return_counts=True)

    mean = np.mean(counts)

    edges = edges[counts>mean]
    
    
    return edges

In [8]:
edges = get_edges(df)
edges

array([[76561197961125336, 76561198113760204],
       [76561197961125336, 76561198170383927],
       [76561197961125336, 76561198294691999],
       ...,
       [76561199065876387, 76561199072921742],
       [76561199067935794, 76561199073762667],
       [76561199070809388, 76561199071222055]], dtype=int64)

In [9]:
# save edges in json file
with open('edges.json', 'w') as f:
    json.dump(edges.tolist(), f)

In [7]:
with open('edges.json', 'r') as f:
    edges = json.load(f)

In [8]:
from netpixi.integration.gt import Graph

g = Graph(directed=False) # não-dirigido

In [9]:
for edge in edges:
    u = int(edge[0])
    v = int(edge[1])
    if not g.has_vertex(u):
        g.add_vertex(u)

    if not g.has_vertex(v):
        g.add_vertex(v)

    if not g.has_edge(u, v):
        g.add_edge(u, v)

In [11]:
print('número de arestas de g:', g.num_edges())

número de arestas de g: 89296


In [12]:
import netpixi
from netpixi.integration.gt import *

In [13]:
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)

In [14]:
gt_save(g, 'reviews_july.net.gz')

In [15]:
r = netpixi.render('reviews_july.net.gz')

In [16]:
r.vertex_default(color=0xD420E9, size=1)

In [17]:
r.edge_default(color=0xFF1401,width=3,curve1=1,curve2=1)